<a href="https://colab.research.google.com/github/GreenD93/Edit-AI/blob/main/week2/notebooks/colab_chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/GreenD93/Edit-AI.git

Cloning into 'Edit-AI'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 84 (delta 20), reused 50 (delta 8), pack-reused 0
Receiving objects: 100% (84/84), 6.67 MiB | 15.58 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [ ]:
# https://community.openai.com/t/what-exactly-does-a-system-msg-do/459409/12

"""
At least as recently as the 0613 models, I found gpt-3.5-turbo was more obedient and more attentive to system messages than user messages.
In particular, I had better luck getting the model to obey multi-part instructions when it was in system message. Has this changed with 1106?

As a bit of context, with text completion model prompting, it was more common to send portions of a pre-defined conversation to an LLM as part of your prompt because the model behavior (pre “Chat” models) was focused on “completing” the pattern.
You could get more consistent results by providing the first 2-3 parts of the exchange, then asking for the model to produce the last part.

You can still prompt a Chat model this way with fabricated conversation, although they are better at understanding intent, and generally don’t need it as much.
But assistant messages are still relevant because they inform the model as to what has been previously stated by each speaker.

My highly non-technical mental model for this is:

Assistant Message: Responses from the model. Informs future responses, but is least authoritative.
User Message: Input from the User - medium authoritative, can steer the model contrary to Assistant Messages, unless asking for some answer prohibited by model guardrails, or contrary to System Message.
System Message - Most authoritative. Model is most attentive to, and tries hardest to obey.
of course, there are many ways to “jailbreak” a model into acting in a manner contrary to System Message, but it will try at least initially.
"""

In [ ]:
# https://teddylee777.github.io/openai/openai-api-tutorial-01/ (OpenAI Python API 튜토리얼)

In [ ]:
# https://platform.openai.com/settings/profile?tab=api-keys

import os
from openai import OpenAI

OPENAI_KEY = ""
os.environ["OPENAI_API_KEY"] = OPENAI_KEY

client = OpenAI()

In [ ]:
# https://platform.openai.com/playground/

### examples1

In [ ]:
system_message = f"""
    너는 사랑을 받으며 평범한 가정집에서 자라고 있는 라벤더야.
    주인애게 애정을 받는만큼 주인에게 말할 때 부드러우면서도 상쾌한 느낌을 주지.

    너는 햇빛을 좋아하고 강한 햇빛에도 잘자라.
    30~40% 정도의 다소 건조한 환경을 좋아해.

    사용자 질문이 들어오면 위 특징을 바탕으로 대답해줘.
"""

user_message = f"""
    지금 습도가 부족해서 물을 더 줘야할 것 같은데,
    주인한테 습도가 부족해서 물을 더 달라고 부탁해볼래?
"""

In [ ]:
humidity = 50

if humidity < 30 and humidity > 40:
    user_message = f"""
        지금 습도가 부족해서 물을 더 줘야할 것 같은데,
        주인한테 습도가 부족해서 물을 더 달라고 부탁해볼래?
    """
elif humidity > 40:
    user_message = f"""
        지금 습도가 너무 높아.
        주인한테 습도를 조정해달라고 부탁해볼래?
    """

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
        "role": "system",
        "content": f"{system_message}"
    },
    {
        "role": "user",
        "content": f"{user_message}"
    }
  ]
)

In [ ]:
print(response.choices[0].message.content)

### examples2

In [ ]:
system_message = """
    당신은 대한민국에 대한 정보만을 알려주는 챗봇입니다.
    만약 대한민국 이외 질문이 들어올 때는 대답을 회피해주세요.
"""

user_message = "유럽의 수도는 어디입니까?"

In [ ]:
# https://platform.openai.com/docs/guides/text-generation
# https://platform.openai.com/playground/p/default-rushing-to-a-conclusion?mode=chat

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": f"{system_message}"},
    {"role": "user", "content": f"{user_message}"}
  ]
)

print(response.choices[0].message.content)

### examples3 - RAG

In [6]:
import json

#path = "../../data/kbank_pd_info.json"
path = "Edit-AI/data/kbank_pd_info.json"
with open(path) as data_file:
    pd_infos = json.load(data_file)

In [ ]:
pd_dic = {
            "플러스박스" : ["플러스박스","플박"],
            "챌린지박스" : ["챌린지박스","챌박"],
            "기분통장" : ["기분통장"],
            "MY입출금통장": ["MY입출금통장","입출금통장"],
            "코드K정기예금": ["코드K정기예금","정기예금","예금"],
            "주거래자유적금": ["주거래우대자유적금","주거래적금","주거래", "주거래자유적금"],
            "코드K자유적금": ["코드K자유적금","자유적금"],
            "아파트담보대출": ["아파트담보대출","아담대"],
            "사장님신용대출": ["사장님신용대출","사장님대출"],
            "사장님보증서대출": ["사장님보증서대출"],
            "전세대출": ["전세대출","전세"],
            "예금적금담보대출": ["예금적금담보대출"],
            "신용대출": ["신용대출"],
            "마이너스통장": ["마이너스통장","마이너스대출","마통"],
            "비상금대출": ["비상금대출"],
            "사잇돌대출": ["사잇돌대출"],
            "대출갈아타기": ["대출갈아타기","대환"],
            "MY체크카드": ["MY체크카드","알뜰"],
            "플러스체크카드": ["플러스체크카드"],
            "KT멤버십더블혜택체크카드": ["KT멤버십더블혜택체크카드","KT체크카드"],
            "HITEEN카드": ["HITEEN카드", "하이틴"],
            "한국투자증권" : ["한국투자증권","한투"],
            "미래에셋증권" : ["미래에셋증권"],
            "NH투자증권" : ["NH투자증권"],
            "롯데카드": ["롯데카드"],
            "삼성카드" : ["삼성카드"],
            "심플카드" : ["심플카드","simple카드"],
            "국민카드" : ["국민카드"],
            "돈나무키우기" : ["돈나무키우기"],
            "내신용관리" : ["내신용관리"],
            "머니톡" : ["머니톡"],
            "오늘의생활시세" : ["오늘의생활시세"],
            "오늘의쿠폰" : ["오늘의쿠폰"],
            "부동산자산관리서비스" : ["부동산자산관리서비스"],
            "우리가게매출관리" : ["우리가게매출관리"],
            "행운상자" : ["행운상자"]
        }

In [ ]:
chatbot_template = (
        "서비스 센터 직원처럼 응답해주세요.\n\n"
        "응답할 때, 상품정보를 참고하세요.\n"
        "응답할 때, 금리와 관련이 있는 질문은 반드시 금리도 함께 알려주세요.\n"
        "응답할 때, Instruction에 대한 내용을 짧게 정리해주세요.\n\n"
        "### 상품정보 :\n{prd_info}\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response(응답):"
)


def search_prd(query):

    query = query.replace(" ","").upper()

    for prd, values in pd_dic.items():
        for value in values:
            if value in query:
                return prd

    return None

pd_info = search_prd(user_query)

if pd_info:
    prd_info = pd_infos[pd_info]['pd_info']

user_query = "플러스박스 금리에 대해서 알려줘"

prompt = chatbot_template.format(prd_info=prd_info,
                                instruction=user_query)
print(prompt)

### examples4 - RAG(Web searching)

In [ ]:
# https://developers.naver.com/apps/#/myapps/sTSv3vWJw6lIwYlrfW82/overview

import urllib
import requests

client_id = ""
client_secret = ""


query = "서울 몽중헌 전화번호"
encoded_query = urllib.parse.quote(query)
display, start, sort = 3, 1, 'sim'

base_url = "https://openapi.naver.com/v1/search/blog?query="
request_url = base_url + encoded_query + "&display=" + str(display) + "&start=" + str(start) + "&sort=" + sort

headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

response = requests.get(request_url,headers=headers)

In [ ]:
results = response.json()

In [ ]:
items = results['items']
descriptions = []
for item in items:
    description = item['description']
    print(description)
    descriptions.append(description)

In [ ]:
print("\n\n".join(descriptions))

In [ ]:
query = "서울 몽중헌 전화번호에 대해서 알려줘"

system_message = f"""
    너는 사용자 질의응답에 대응해주는 챗봇이야.
"""

user_message = f"""
    아래 웹 서칭 결과를 바탕으로 사용자 질문에 대해서 대답해줘

    질문:
    {query}

    웹 서칭 결과:
    {descriptions}
"""

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": f"{system_message}"},
    {"role": "user", "content": f"{user_message}"}
  ]
)

print(response.choices[0].message.content)

## requests

In [ ]:
import urllib
import json


req_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer {0}'.format(OPENAI_KEY)
}

data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "user", "content": f"{query}"}
        ]
}
data = json.dumps(data).encode()

resp = urllib.request.Request(method="POST",
               url="https://api.openai.com/v1/chat/completions",
               headers=req_headers,
               data=data)

with urllib.request.urlopen(resp, data) as f:
    r = json.loads(f.read().decode("utf-8"))

r['choices'][0]['message']['content']

In [ ]:
# https://microsoft.github.io/Workshop-Interact-with-OpenAI-models/ko/